In [1]:
import requests
import pandas as pd

# Actual total load per bidding zone per Settlement Period, the total load being defined as equal
# to the sum of power generated by plants on both Transmission System Operators/Distribution System
# Operators networks, from which is deduced:
# - The balance (export-import) of exchanges on interconnections between neighbouring bidding zones.
# - The power absorbed by energy storage resources.

# demand data is published +1H after
end_date = pd.Timestamp.now()
settlementDate = (end_date - pd.Timedelta(hours=0.5)).date()
settlementPeriod = (end_date - pd.Timedelta(hours=0.5)).hour * 2

from_ = (end_date).date()
to_ = from_

web = 'https://data.elexon.co.uk/bmrs/api/v1/demand/actual/total?from=' + str(from_) + '&to=' + str(to_) + '&settlementPeriodFrom=' + str(settlementPeriod) + '&settlementPeriodTo=' + str(settlementPeriod) + '&format=json'
r = requests.get(web)
df = pd.DataFrame(r.json()['data'])
df

,publishTime,startTime,settlementDate,settlementPeriod,quantity
0,2024-02-02T12:25:09Z,2024-02-02T11:30:00Z,2024-02-02,24,47077.0


In [1]:
import sys
import pandas as pd
from pathlib import Path

sys.path.append(str(Path.cwd().parent / 'scripts'))

from _aggregation_helpers import get_demand

In [18]:
file = Path.cwd().parent / 'data' / "elexon_profiling_data_201314.xlsx"


index_mapper = {
    'domestic single': slice(0,48),
    'domestic multi': slice(52,100),
    'non-domestic single': slice(104,152),
    'non-domestic multi': slice(156,204),
}

single_rate_domestic = pd.read_excel(
    file,
    index_col=0,
    header=2).iloc[index_mapper['domestic single']]
multi_rate_domestic = pd.read_excel(
    file,
    index_col=0,
    header=2).iloc[index_mapper['domestic multi']]

single_rate_nondomestic = pd.read_excel(
    file,
    index_col=0,
    header=2).iloc[index_mapper['non-domestic single']]
multi_rate_nondomestic = pd.read_excel(
    file,
    index_col=0,
    header=2).iloc[index_mapper['non-domestic multi']]

ps = range(1, 49)
dates = pd.date_range(start='2023-07-12', end='2024-07-11', freq='D')



from itertools import product

sum(list(map(
    lambda args: get_demand(*args),
    list(product([multi_rate_nondomestic], dates, ps))))
    )

51363.83769137823

In [31]:
srd = 8421.5 / 2
mrd = 10990.3 / 2
srn = 33333.8 / 2
mrn = 51364.8 / 2

print(srd, mrd, srn, mrn)

4210.75 5495.15 16666.9 25682.4
